# Fine Tune Com Wikipédia

### Descriçao

---
1. O objetivo deste notebook é aprender sobre Fine Tune
2. Vamos descorrer sobre modelos de LLM e utilizar a biblioteca Torch para descorrer sobre assunto
3. Vamos ensinar nossa máquina com os dados do [Wikipédia](https://pt.wikipedia.org/wiki/Ci%C3%AAncia)
---

### Baixando os pacotes

In [1]:
!pip install transformers transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

### importando as bibliotecas

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import requests
from datasets import Dataset, load_dataset
import torch

### importando o modelo

In [3]:
model_name = "pierreguillou/gpt2-small-portuguese"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

# Requisitando o Wikipedia

In [5]:


url = "https://pt.wikipedia.org/wiki/Ci%C3%AAncia"
response = requests.get(url)
text_data = response.text

lines = text_data.split('\n')
data_dict = {"text": lines}

dataset = Dataset.from_dict(data_dict)

In [6]:
# tokenização do dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1975 [00:00<?, ? examples/s]

In [8]:
first_example = tokenized_dataset[1250]
print(first_example)
print(first_example['text'])

{'text': '<figure class="mw-default-size" typeof="mw:File/Thumb"><a href="/wiki/Ficheiro:2021_Survey_on_existence_of_global_warming_and_responsibility_for_climate_change_-_bar_chart.svg" class="mw-file-description"><img alt="Result in bar graph of two questions (&quot;Is global warming occurring?&quot; and &quot;Are oil/gas companies responsible?&quot;), showing large discrepancies between American Democrats and Republicans" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/65/2021_Survey_on_existence_of_global_warming_and_responsibility_for_climate_change_-_bar_chart.svg/220px-2021_Survey_on_existence_of_global_warming_and_responsibility_for_climate_change_-_bar_chart.svg.png" decoding="async" width="220" height="124" class="mw-file-element" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/6/65/2021_Survey_on_existence_of_global_warming_and_responsibility_for_climate_change_-_bar_chart.svg/330px-2021_Survey_on_existence_of_global_warming_and_responsibility_for_climate_change

In [9]:
data_collator = DataCollatorForLanguageModeling( tokenizer=tokenizer,mlm=False)

In [10]:
# hiper parâmetros
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1, # !!!
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2
)

In [11]:
trainer = Trainer(model=model,args=training_args, data_collator=data_collator, train_dataset=tokenized_dataset)

In [12]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=62, training_loss=2.496186287172379, metrics={'train_runtime': 65.382, 'train_samples_per_second': 30.207, 'train_steps_per_second': 0.948, 'total_flos': 129012940800000.0, 'train_loss': 2.496186287172379, 'epoch': 1.0})

### Salvando o modelo

In [13]:
model.save_pretrained("meu_modelo_treinado")
tokenizer.save_pretrained("meu_modelo_treinado")

('meu_modelo_treinado/tokenizer_config.json',
 'meu_modelo_treinado/special_tokens_map.json',
 'meu_modelo_treinado/vocab.json',
 'meu_modelo_treinado/merges.txt',
 'meu_modelo_treinado/added_tokens.json')

# Utilizando o Modelo

#importando a biblioteca que criamos

In [14]:
from transformers import pipeline

In [15]:
gerador = pipeline("text-generation", model="meu_modelo_treinado")
texto = "Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico."
resultado = gerador(texto, max_length=60, do_sample=True, temperature=0.7, top_k=50, top_p=0.85)
print(resultado)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico. A ciência é a ciência que estuda e estuda o conhecimento.\n\nAs ciências são as ciências que estudam e estudam o conhecimento. A ciência de engenharia é a ciência que estuda e estuda o conhecimento. A ciência'}]
